In [12]:
import numpy as np
import math, time, sys
from PIL import Image
import cv2


def arnoldscramble(img):
    lena = img

    # Arnold Transform Parameters
    a = 6
    b = 40
    rounds = 33

    # Open the images
    

    print(" ~~~~~~  * PARAMETERS *  ~~~~~~ ")
    arnold = Arnold(a, b, rounds)
    print("\ta:\t", a)
    print("\tb:\t", b)
    print("\trounds:\t", rounds)

    print("\n ~~~~~~  *  RESULTS   *  ~~~~~~ ")
    
    start_time = time.time()
    scrambled = Arnold.applyTransformTo(arnold, lena)
    exec_time = time.time() - start_time
    print("Transform  execution time: %.6f " % exec_time, "sec")
    
    cv2.imwrite('scrambled.png',scrambled)

    start_time = time.time()
    reconstructed = Arnold.applyInverseTransformTo(arnold, scrambled)
    exec_time = time.time() - start_time
    print("Inverse T. execution time: %.6f " % exec_time, "sec")
    im = Image.fromarray(reconstructed).convert("L")
    cv2.imwrite('reconstructed.png',reconstructed)

    counter = 0
    for i in range(scrambled.shape[0]):
        for j in range(scrambled.shape[0]):
            if(lena[i, j] != reconstructed[i, j]):
                print(lena[i, j], " != ", reconstructed[i, j])
                counter += 1
    print("\nDIFFERENT PIXELS\n\toriginal  VS reconstructed:\t\t", counter)


'''if __name__ == "__main__":
    main(sys.argv[1:])'''

'if __name__ == "__main__":\n    main(sys.argv[1:])'

In [13]:
img = cv2.imread("TestImages/lena.png",cv2.IMREAD_GRAYSCALE)

In [14]:
arnoldscramble(img)

 ~~~~~~  * PARAMETERS *  ~~~~~~ 
	a:	 6
	b:	 40
	rounds:	 33

 ~~~~~~  *  RESULTS   *  ~~~~~~ 
Transform  execution time: 0.177538  sec
Inverse T. execution time: 0.172992  sec

DIFFERENT PIXELS
	original  VS reconstructed:		 0


In [5]:
class Arnold:

    def __init__(self, a:int, b:int, rounds:int):
        # Parameters
        self.__a = a
        self.__b = b
        self.__rounds = rounds

    def mapping(self, s:np.shape):
        x, y = np.meshgrid(range(s[0]), range(s[0]), indexing="ij")
        xmap = (self.__a*self.__b*x + x + self.__a*y) % s[0]
        ymap = (self.__b*x + y) % s[0]
        return xmap, ymap

    def inverseMapping(self, s:np.shape):
        x, y = np.meshgrid(range(s[0]), range(s[0]), indexing="ij")
        xmap = (x - self.__a*y) % s[0]
        ymap = (-self.__b*x + self.__a*self.__b*y + y) % s[0]
        return xmap, ymap

    def applyTransformTo(self, image:np.ndarray):
        xm, ym = self.mapping(image.shape)
        img = image
        for r in range(self.__rounds):
            img = img[xm, ym]
        return img

    def applyInverseTransformTo(self, image:np.ndarray):
        xm, ym = self.inverseMapping(image.shape)
        img = image
        for r in range(self.__rounds):
            img = img[xm, ym]
        return img